In [ ]:
#Librería para manipular datos
import pandas as pd
# llamar a la API de Zendesk
import numpy as np
import requests
import time
import json
import base64
import glob
import os
from os import walk
from google.colab import files
#@markdown 
#config
pd.set_option('display.max_columns', None)

In [ ]:
#______________________________________________________________________________________
#@markdown
Dominio = '' #@param {type: "string"}

Correo = '' #@param {type: "string"}
Contraseña = '' #@param {type: "string"}
Tipo_Usuario = "Usuario Final " #@param [" Agente", "Usuario Final "]


##agrega el subdominio d el acuenta de zendesk de la cual importaras los
##usuarios /leyendas

z_domain=Dominio
email=Correo
contraseña=Contraseña
rol= '360009592011'
#tipo_usuario=Tipo_usuario

authentication=email+"/token:"+contraseña
message_bytes = authentication.encode('ascii')
base64_bytes=base64.b64encode(message_bytes).decode('ascii')
headers = {'Authorization': 'Basic '+base64_bytes, 'Content-Type': 'application/json'}

In [ ]:
from google.colab import files
uploaded = files.upload()


In [ ]:
with open('/content/usuarios_peigo_2022_11_22_part_05.csv', encoding="ISO-8859-1") as csv_file:
  df=pd.read_csv(csv_file,sep="," )


df["custom_fields.ciudad"] = df["custom_fields.ciudad"].fillna("")
df["custom_fields.nacionalidad"] = df["custom_fields.nacionalidad"].fillna("")
df["custom_fields.provincia"] = df["custom_fields.provincia"].fillna("")
nan_cols = [i for i in df.columns if df[i].isnull().any()]
nan_cols
#df.dtypes

In [ ]:
url = z_domain+"/api/v2/users/create_or_update"
resumen = []

for i in range(len(df)):
 
  data=json.dumps({
  "user": {
    "name": str(df.iloc[i]["name"]),
    "email": str(df.iloc[i]["email"]),
    "phone": "+"+str(df.iloc[i]["phone"]),
    "verified": True,
    "active": True,
    "user_fields": {
      "cedula": int(df.iloc[i]["custom_fields.cedula"]),
      "ciudad": str(df.iloc[i]["custom_fields.ciudad"]),
      "embajador": bool(df.iloc[i]["custom_fields.embajador"]),
      "empleado_del_banco": bool(df.iloc[i]["custom_fields.empleado_del_banco"]),
      "genero": str(df.iloc[i]["custom_fields.genero"]),
      "nacionalidad": df.iloc[i]["custom_fields.nacionalidad"],
      "provincia": str(df.iloc[i]["custom_fields.provincia"]),
      "segmento": str(df.iloc[i]["custom_fields.segmento"]),
      "vp_del_banco": bool(df.iloc[i]["custom_fields.vp_del_banco"])
    }
  }
})
  r = requests.post(url = url, headers = headers, data = data)
  print(df.iloc[i]["name"],r.status_code,r.content)
  resumen.append([df.iloc[i]["name"],r.status_code,r.content])
  time.sleep(5)

rr = pd.DataFrame(resumen,columns=["resultado","status_code","respuesta"])

rr.to_csv("/content/estado_carga.csv")

files.download("/content/estado_carga.csv")
